In [1]:
# Import Packages
import pandas as pd, numpy as np
from pathlib import Path
import os, sys, glob, re
from collections import OrderedDict
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

# Import Board Summary

In [2]:
inputfilepath = Path("Board Summary.csv")

In [3]:
df = pd.read_csv(inputfilepath, low_memory=False)
# Replace NaNs with 0
# df = df.replace('n.a', np.nan)
# df = df.fillna(0)

# Get month and year
df['Month'] = df['Annual Report Year'].str[0:4]
df['Year'] = df['Annual Report Year'].str[4:]

df

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,...,Director Experience - Avg. Yrs on Other Quoted Boards,Characteristics of Roles - Individual Role,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,...,6.1,Independent NED,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
1,0.0,NaN,Current,NaN,SD,1.2,NaN,NaN,NaN,NaN,Samantha (Sam) Jane Hoe-Richardson,NaN,2.0,NaN,NaN,13534906790,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,4.0,9811997164,...,8.2,Independent NED,NaN,1486,NaN,NaN,NaN,5.0,19.8,0.0,JE00BF5FX167,NaN,1.2,Investment Companies,2.0,1.2,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
2,1.0,NaN,Current,NaN,SD,8.0,NaN,NaN,NaN,NaN,Paul A Masterton,NaN,3.0,NaN,NaN,13584596849,NaN,NaN,0.0,NaN,NaN,FTSE 250,3IN,5.0,9811997164,...,0.0,Senior Independent NED,NaN,1509,NaN,NaN,NaN,100.0,NaN,0.0,JE00BF5FX167,NaN,8.0,Investment Companies,1.0,4.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
3,NaN,NaN,Current,NaN,SD,6.9,NaN,NaN,NaN,NaN,Ian Matthew Lobley,NaN,4.0,NaN,NaN,13665216944,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,20.0,9811997164,...,0.0,NED,NaN,1129,NaN,NaN,NaN,NaN,11.8,0.0,JE00BF5FX167,NaN,6.9,Investment Companies,1.0,6.9,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
4,1.0,NaN,Current,NaN,SD,6.3,NaN,NaN,NaN,NaN,James (Doug) Douglas Bannister,NaN,NaN,NaN,NaN,15379548720,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,...,0.0,Independent NED,NaN,64,NaN,NaN,NaN,68.0,13.8,0.0,JE00BF5FX167,NaN,6.3,Investment Companies,1.0,6.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568524,NaN,NaN,Jun 2016,NaN,SM,NaN,NaN,NaN,NaN,NaN,Campbell Hair,NaN,NaN,NaN,NaN,13637986908,NaN,NaN,NaN,NaN,NaN,NaN,XCH,NaN,8113811537,...,NaN,Global HR Director,NaN,235,NaN,NaN,NaN,NaN,NaN,NaN,GB00B1VK7X76,NaN,NaN,Business Services,NaN,NaN,NaN,UK - SMDEs Org Summary,UK,SMDE,SMDEs Org Summary,NaN,NaN,Jun,2016
1568525,NaN,NaN,Jun 2016,NaN,SM,NaN,NaN,NaN,NaN,NaN,James (Jim) Anthony Sadler,NaN,NaN,NaN,NaN,13638046908,NaN,NaN,NaN,NaN,NaN,NaN,XCH,NaN,8113811537,...,NaN

In [4]:
# Get current years
# Replace 'Current' with the latest month (previous month of company + 1 year)
current_year = (df[df['Annual Report Year'] != 'Current'].groupby('CompanyID*')['Year'].max().astype('int') + 1).reset_index()
current_year

,CompanyID*,Year
0,270810,2021
1,386710,2020
2,386770,2012
3,422759,2009
4,493378,2021
...,...,...
3242,305172912690,2021
3243,305214912690,2021
3244,305299712697,2020
3245,306519112781,2020


In [5]:
# Get current months
current_month = df[df['Annual Report Year'] != 'Current'].groupby('CompanyID*')['Month'].first().reset_index()
current_month

,CompanyID*,Month
0,270810,Dec
1,386710,Dec
2,386770,Dec
3,422759,Dec
4,493378,Dec
...,...,...
3242,305172912690,Jun
3243,305214912690,Dec
3244,305299712697,Dec
3245,306519112781,Dec


In [6]:
# Get current year and month
current_year_month = current_year.merge(current_month, on='CompanyID*', how='inner')
current_year_month = current_year_month.rename(columns={'Year':'Current Year', 'Month':'Current Month'})
current_year_month

,CompanyID*,Current Year,Current Month
0,270810,2021,Dec
1,386710,2020,Dec
2,386770,2012,Dec
3,422759,2009,Dec
4,493378,2021,Dec
...,...,...,...
3242,305172912690,2021,Jun
3243,305214912690,2021,Dec
3244,305299712697,2020,Dec
3245,306519112781,2020,Dec


In [7]:
# Merge current year and month back to df
df = df.merge(current_year_month, on=['CompanyID*'], how='inner')

In [8]:
df['Year'] = np.where(df['Annual Report Year'] == 'Current', df['Current Year'], df['Year'])
df['Month'] = np.where(df['Annual Report Year'] == 'Current', df['Current Month'], df['Month'])
df

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,...,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year,Current Year,Current Month
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,...,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
1,0.0,NaN,Current,NaN,SD,1.2,NaN,NaN,NaN,NaN,Samantha (Sam) Jane Hoe-Richardson,NaN,2.0,NaN,NaN,13534906790,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,4.0,9811997164,...,NaN,1486,NaN,NaN,NaN,5.0,19.8,0.0,JE00BF5FX167,NaN,1.2,Investment Companies,2.0,1.2,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
2,1.0,NaN,Current,NaN,SD,8.0,NaN,NaN,NaN,NaN,Paul A Masterton,NaN,3.0,NaN,NaN,13584596849,NaN,NaN,0.0,NaN,NaN,FTSE 250,3IN,5.0,9811997164,...,NaN,1509,NaN,NaN,NaN,100.0,NaN,0.0,JE00BF5FX167,NaN,8.0,Investment Companies,1.0,4.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
3,NaN,NaN,Current,NaN,SD,6.9,NaN,NaN,NaN,NaN,Ian Matthew Lobley,NaN,4.0,NaN,NaN,13665216944,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,20.0,9811997164,...,NaN,1129,NaN,NaN,NaN,NaN,11.8,0.0,JE00BF5FX167,NaN,6.9,Investment Companies,1.0,6.9,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
4,1.0,NaN,Current,NaN,SD,6.3,NaN,NaN,NaN,NaN,James (Doug) Douglas Bannister,NaN,NaN,NaN,NaN,15379548720,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,...,NaN,64,NaN,NaN,NaN,68.0,13.8,0.0,JE00BF5FX167,NaN,6.3,Investment Companies,1.0,6.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568478,NaN,NaN,Jun 2016,NaN,SM,NaN,NaN,NaN,NaN,NaN,Campbell Hair,NaN,NaN,NaN,NaN,13637986908,NaN,NaN,NaN,NaN,NaN,NaN,XCH,NaN,8113811537,...,NaN,235,NaN,NaN,NaN,NaN,NaN,NaN,GB00B1VK7X76,NaN,NaN,Business Services,NaN,NaN,NaN,UK - SMDEs Org Summary,UK,SMDE,SMDEs Org Summary,NaN,NaN,Jun,2016,2017,Dec
1568479,NaN,NaN,Jun 2016,NaN,SM,NaN,NaN,NaN,NaN,NaN,James (Jim) Anthony Sadler,NaN,NaN,NaN,NaN,13638046908,NaN,NaN,NaN,NaN,NaN,NaN,XCH,NaN,8113811537,...,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,GB00B1VK7X76,NaN,NaN,Business Services,NaN,NaN,NaN,UK - SMDEs Org Summary,UK,SMDE,SMDEs Org Summary,NaN,N

In [9]:
# View dtypes and change if necessary
df.dtypes

Ratios - Wealth Delta                                                      float64
Total - Total Annual Compensation                                          float64
Annual Report Year                                                          object
Annual- Equity Linked Options - Intrinsic Options (excercisable)           float64
Director Type                                                               object
Characteristics of Roles - Time on Board                                   float64
Accumulated Wealth - LTIPS(max)                                            float64
Ratios - Bonus/ (Bonus&Salary)                                             float64
Annual- Equity Linked Options - Total Equity Linked Compensation           float64
Annual- Equity Linked Options - Share Price                                float64
Individual Name                                                             object
Annual- Equity Linked Options - LTIPS(max)                                 float64
Dire

In [10]:
# Change data types to enable .groupby to work later on
data_types_dict = {'Year': 'int64'}
df = df.astype(data_types_dict)
df.dtypes

Ratios - Wealth Delta                                                      float64
Total - Total Annual Compensation                                          float64
Annual Report Year                                                          object
Annual- Equity Linked Options - Intrinsic Options (excercisable)           float64
Director Type                                                               object
Characteristics of Roles - Time on Board                                   float64
Accumulated Wealth - LTIPS(max)                                            float64
Ratios - Bonus/ (Bonus&Salary)                                             float64
Annual- Equity Linked Options - Total Equity Linked Compensation           float64
Annual- Equity Linked Options - Share Price                                float64
Individual Name                                                             object
Annual- Equity Linked Options - LTIPS(max)                                 float64
Dire

In [11]:
# Only only get data from 2002 onwards
df = df[df['Year'] >= 2002]
df

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,...,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year,Current Year,Current Month
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,...,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
1,0.0,NaN,Current,NaN,SD,1.2,NaN,NaN,NaN,NaN,Samantha (Sam) Jane Hoe-Richardson,NaN,2.0,NaN,NaN,13534906790,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,4.0,9811997164,...,NaN,1486,NaN,NaN,NaN,5.0,19.8,0.0,JE00BF5FX167,NaN,1.2,Investment Companies,2.0,1.2,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
2,1.0,NaN,Current,NaN,SD,8.0,NaN,NaN,NaN,NaN,Paul A Masterton,NaN,3.0,NaN,NaN,13584596849,NaN,NaN,0.0,NaN,NaN,FTSE 250,3IN,5.0,9811997164,...,NaN,1509,NaN,NaN,NaN,100.0,NaN,0.0,JE00BF5FX167,NaN,8.0,Investment Companies,1.0,4.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
3,NaN,NaN,Current,NaN,SD,6.9,NaN,NaN,NaN,NaN,Ian Matthew Lobley,NaN,4.0,NaN,NaN,13665216944,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,20.0,9811997164,...,NaN,1129,NaN,NaN,NaN,NaN,11.8,0.0,JE00BF5FX167,NaN,6.9,Investment Companies,1.0,6.9,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
4,1.0,NaN,Current,NaN,SD,6.3,NaN,NaN,NaN,NaN,James (Doug) Douglas Bannister,NaN,NaN,NaN,NaN,15379548720,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,...,NaN,64,NaN,NaN,NaN,68.0,13.8,0.0,JE00BF5FX167,NaN,6.3,Investment Companies,1.0,6.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568478,NaN,NaN,Jun 2016,NaN,SM,NaN,NaN,NaN,NaN,NaN,Campbell Hair,NaN,NaN,NaN,NaN,13637986908,NaN,NaN,NaN,NaN,NaN,NaN,XCH,NaN,8113811537,...,NaN,235,NaN,NaN,NaN,NaN,NaN,NaN,GB00B1VK7X76,NaN,NaN,Business Services,NaN,NaN,NaN,UK - SMDEs Org Summary,UK,SMDE,SMDEs Org Summary,NaN,NaN,Jun,2016,2017,Dec
1568479,NaN,NaN,Jun 2016,NaN,SM,NaN,NaN,NaN,NaN,NaN,James (Jim) Anthony Sadler,NaN,NaN,NaN,NaN,13638046908,NaN,NaN,NaN,NaN,NaN,NaN,XCH,NaN,8113811537,...,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,GB00B1VK7X76,NaN,NaN,Business Services,NaN,NaN,NaN,UK - SMDEs Org Summary,UK,SMDE,SMDEs Org Summary,NaN,N

In [12]:
# Prepare dataset to get variables for non-director related variables
# 1 row = 1 firm for 1 year
df1 = df.drop_duplicates(subset=['Year', 'Month', 'CompanyID*'], ignore_index=True)
df1

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,...,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year,Current Year,Current Month
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,...,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
1,NaN,NaN,Mar 2007,NaN,SD,0.0,NaN,NaN,NaN,NaN,Peter Wagner,NaN,4.0,0.0,NaN,46356308,NaN,NaN,0.0,NaN,Swiss,FTSE 250,3IN,21.0,9811997164,...,NaN,304,NaN,NaN,NaN,NaN,9.1,1.0,JE00BF5FX167,NaN,0.0,Investment Companies,2.0,0.0,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2007,2021,Mar
2,1.0,132.0,Mar 2008,NaN,SD,1.0,NaN,NaN,NaN,NaN,Peter Wagner,NaN,3.0,132.0,NaN,46356308,NaN,NaN,132.0,NaN,Swiss,FTSE 250,3IN,21.0,9811997164,...,NaN,304,NaN,NaN,NaN,55.0,8.1,1.0,JE00BF5FX167,NaN,1.0,Investment Companies,2.0,1.0,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2008,2021,Mar
3,NaN,12.0,Mar 2009,NaN,SD,0.3,NaN,NaN,NaN,NaN,Sir John Alexander Collins,NaN,1.0,12.0,NaN,127555791,NaN,NaN,12.0,NaN,British,FTSE 250,3IN,6.0,9811997164,...,NaN,952,NaN,NaN,NaN,NaN,2.8,1.0,JE00BF5FX167,NaN,0.3,Investment Companies,2.0,0.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2009,2021,Mar
4,NaN,26.0,Mar 2010,NaN,SD,0.5,NaN,NaN,NaN,NaN,Kirsten Charlotte Valeur-Adu,NaN,29.0,26.0,1.0,11225133228,NaN,NaN,26.0,NaN,Danish,FTSE 250,3IN,29.0,9811997164,...,NaN,2207,1.0,NaN,NaN,NaN,23.8,0.0,JE00BF5FX167,NaN,0.5,Investment Companies,2.0,0.5,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2010,2021,Mar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45432,4.0,455.0,Dec 2011,NaN,ED,0.5,304.0,0.26,224.0,0.65,David Gregory Bauernfeind,184.0,NaN,172.0,NaN,11717154218,NaN,40.0,232.0,NaN,British,NaN,XCH,NaN,8113811537,...,NaN,0,NaN,NaN,7.0,42.0,21.3,1.0,GB00B1VK7X76,0.49,4.6,Business Services,1.0,0.5,M,UK - Board Summary,UK,Director,Board Summary,NaN,NaN,Dec,2011,2017,Dec
45433,15.0,1168.0,Dec 2012,NaN,ED,1.5,1160.0,0.50,573.0,NaN,David Gregory Bauernfeind,573.0,NaN,300.0,NaN,11717154218,46.0,77.0,595.0,NaN,British,NaN,XCH,NaN,8113811537,...,NaN,0,NaN,NaN,9.0,200.0,20.3,1.0,GB00B1VK7X76,0.49,5.6,Business Services,1.0,1.5,M,UK - Board Summary,UK,Dire

In [13]:
# View columns
df.columns

Index(['Ratios - Wealth Delta', 'Total - Total Annual Compensation',
       'Annual Report Year',
       'Annual- Equity Linked Options - Intrinsic Options (excercisable)',
       'Director Type', 'Characteristics of Roles - Time on Board',
       'Accumulated Wealth - LTIPS(max)', 'Ratios - Bonus/ (Bonus&Salary)',
       'Annual- Equity Linked Options - Total Equity Linked Compensation',
       'Annual- Equity Linked Options - Share Price', 'Individual Name',
       'Annual- Equity Linked Options - LTIPS(max)',
       'Director Experience - Total Number of Private Current Boards',
       'Annual Direct Compensation - Salary',
       'Director Experience - Total Number of Other Current Boards',
       'DirectorID*', 'Accumulated Wealth - Intrinsic Option',
       'Accumulated Wealth - Estimated Option',
       'Annual Direct Compensation - Total Salary+Bonus',
       'Annual- Equity Linked Options - Shares',
       'Director Experience - Nationality Mix', 'Index', 'Ticker',
       'Dir

# Groupby year and company

In [14]:
def create_agg_dict(agg_list, function):
    agg_dict = {}
    for var in agg_list:
        agg_dict[var] = function
    return agg_dict

In [15]:
agg_list_first = ['Country', 'ISIN', 'region']
agg_list_mean = ['Characteristics of Roles - Director Network Size', 'Characteristics of Roles - Time to Retirement', 
                 'Characteristics of Roles - Time in Role', 'Characteristics of Roles - Time on Board', 
                 'Characteristics of Roles - Time in Company',
                 'Director Experience - Total Number of Quoted Boards to Date', 'Director Experience - Total Number of Private Boards to Date', 
                 'Director Experience - Total Number of Other Boards to Date', 
                 'Director Experience - Total Number of Quoted Current Boards', 'Director Experience - Total Number of Private Current Boards', 
                 'Director Experience - Total Number of Other Current Boards', 
                 'Director Experience - Avg. Yrs on Other Quoted Boards', 'Director Experience - Age (Yrs)', 
                 'Director Experience - Number of Qualifications',
                 'Ratios - Bonus/ (Bonus&Salary)','Ratios - Equity Linked/ Total', 
                 'Ratios - Performance/ Total', 'Ratios - %Change from Last Period', 'Ratios - Wealth Delta',
                 'Director Count Totals - Number of Independent NED with past CFO/FD role',
                 'Annual Direct Compensation - Salary',  'Annual Direct Compensation - Bonus', 
                 'Annual Direct Compensation - D.C Pension', 'Annual Direct Compensation - Other', 
                 'Annual Direct Compensation - Total Salary+Bonus', 
                 'Annual Direct Compensation - Total Inc. D.C. Pension & Other',
                 'Annual- Equity Linked Options - Shares', 'Annual- Equity Linked Options - LTIPS(max)',
                 'Annual- Equity Linked Options - Intrinsic Options (excercisable)',  
                 'Annual- Equity Linked Options - Intrinsic Options (unexercisable)',
                 'Annual- Equity Linked Options - Estimated Options (unexcercisable)',
                 'Annual- Equity Linked Options - Estimated Options (exercisable)',
                 'Annual- Equity Linked Options - Share Price', 'Annual- Equity Linked Options - Total Equity Linked Compensation', 
                 'Total - Total Annual Compensation',
                 'Accumulated Wealth - Shares', 'Accumulated Wealth - LTIPS(max)', 
                 'Accumulated Wealth - Intrinsic Option', 'Accumulated Wealth - Estimated Option', 
                 'Accumulated Wealth - Liquid Wealth', 
                 'Total Wealth - Total Wealth']
agg_list_list = ['Director Experience - Nationality Mix']

agg_dict_first = create_agg_dict(agg_list_first, 'first')
agg_dict_mean = create_agg_dict(agg_list_mean, 'mean')
agg_dict_list = create_agg_dict(agg_list_list, list)

agg_dict = {**agg_dict_first, **agg_dict_mean}
# agg_dict = {**agg_dict_first, **agg_dict_mean, **agg_dict_list, **agg_dict_udf}
agg_dict = OrderedDict(agg_dict)
agg_dict

OrderedDict([('Country', 'first'),
             ('ISIN', 'first'),
             ('region', 'first'),
             ('Characteristics of Roles - Director Network Size', 'mean'),
             ('Characteristics of Roles - Time to Retirement', 'mean'),
             ('Characteristics of Roles - Time in Role', 'mean'),
             ('Characteristics of Roles - Time on Board', 'mean'),
             ('Characteristics of Roles - Time in Company', 'mean'),
             ('Director Experience - Total Number of Quoted Boards to Date',
              'mean'),
             ('Director Experience - Total Number of Private Boards to Date',
              'mean'),
             ('Director Experience - Total Number of Other Boards to Date',
              'mean'),
             ('Director Experience - Total Number of Quoted Current Boards',
              'mean'),
             ('Director Experience - Total Number of Private Current Boards',
              'mean'),
             ('Director Experience - Total Number

In [16]:
# See how many year-months a company has
print(df.groupby(['CompanyID*', 'Company Name', 'Sector'])['Annual Report Year'].nunique().max())
print(df.groupby(['CompanyID*', 'Company Name', 'Sector'])['Year'].nunique().max())
# This means that each company only has 1 year-month a year.
# 2022 - 2002 + 1 = 21

21
21


In [17]:
# Note: CompanyID and CompanyID + Company Name both gave same number of rows
agg_dict_final = dict(list(agg_dict.items())[:]) # subset for debugging
# First we aggregate to the year-month level
df_groupby_yearmonth = df.groupby(['Year', 'Month', 'CompanyID*', 'Company Name', 'Sector']).agg(agg_dict_final).reset_index()
# Then we aggregate to the year level
# This gives equal weight to each month
df_groupby = df_groupby_yearmonth.groupby(['Year', 'CompanyID*', 'Company Name', 'Sector']).agg(agg_dict_final).reset_index()
df_groupby

,Year,CompanyID*,Company Name,Sector,Country,ISIN,region,Characteristics of Roles - Director Network Size,Characteristics of Roles - Time to Retirement,Characteristics of Roles - Time in Role,Characteristics of Roles - Time on Board,Characteristics of Roles - Time in Company,Director Experience - Total Number of Quoted Boards to Date,Director Experience - Total Number of Private Boards to Date,Director Experience - Total Number of Other Boards to Date,Director Experience - Total Number of Quoted Current Boards,Director Experience - Total Number of Private Current Boards,Director Experience - Total Number of Other Current Boards,Director Experience - Avg. Yrs on Other Quoted Boards,Director Experience - Age (Yrs),Director Experience - Number of Qualifications,Ratios - Bonus/ (Bonus&Salary),Ratios - Equity Linked/ Total,Ratios - Performance/ Total,Ratios - %Change from Last Period,Ratios - Wealth Delta,Director Count Totals - Number of Independent NED with past CFO/FD role,Annual Direct Compensation - Salary,Annual Direct Compensation - Bonus,Annual Direct Compensation - D.C Pension,Annual Direct Compensation - Other,Annual Direct Compensation - Total Salary+Bonus,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Shares,Annual- Equity Linked Options - LTIPS(max),Annual- Equity Linked Options - Intrinsic Options (excercisable),Annual- Equity Linked Options - Intrinsic Options (unexercisable),Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Estimated Options (exercisable),Annual- Equity Linked Options - Share Price,Annual- Equity Linked Options - Total Equity Linked Compensation,Total - Total Annual Compensation,Accumulated Wealth - Shares,Accumulated Wealth - LTIPS(max),Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Accumulated Wealth - Liquid Wealth,Total Wealth - Total Wealth
0,2002,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,570.578947,7.857143,7.466667,9.972727,12.200000,1.545455,4.666667,1.0,1.363636,1.888889,1.0,2.575000,59.428571,2.133333,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,493378,FIRST COMMONWEALTH FINANCIAL CORP PA,Banks,United States,US3198291078,None,295.909091,6.400000,6.376471,10.868750,11.835294,1.125000,7.882353,NaN,1.000000,3.625000,NaN,0.000000,60.764706,1.176471,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,497536,FIRST COMMUNITY BANKSHARES INC (First Communit...,Banks,United States,US31983A1034,None,402.076923,9.269231,7.861538,11.500000,11.053846,1.000000,5.416667,1.0,1.000000,1.750000,1.0,0.000000,58.307692,2.076923,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,7012097,FIRST INDUSTRIAL REALTY TRUST INC,Real Estate,United States,US32054K1034,None,1920.105263,14.364286,5.921429,7.670000,7.835714,2.800000,4.100000,NaN,1.600000,1.500000,NaN,3.220000,52.571429,1.857143,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,7168536,BENCHMARK ELECTRONICS INC,Electronic & Electrical Equipment,United States,US08160H1014,None,455.000000,10.900000,5.020000,12.360000,15.450000,1.111111,2.000000,NaN,1.000000,NaN,NaN,1.100000,55.100000,2.200000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45167,2022,3584311110,SLEEP NUMBER CORP (Select Comfort Corp prior t...,Household Products,United States,US83125X1037,None,1862.950000,8.140000,8.860000,10.460000,10.940000,2.600000,2.714286,NaN,1.800000,1.250000,NaN,5.080000,61.500000,2.200000,NaN,NaN,NaN,

# Get shares by gender and nationality

In [18]:
# Get shares by gender
df_charac = pd.DataFrame(df.groupby(['CompanyID*', 'Year', 'Director Experience - Gender'])['Director Experience - Gender'].count())
df_charac.columns = ['count']
df_charac = df_charac.reset_index()
df_charac = df_charac.pivot(index=['CompanyID*', 'Year'], columns='Director Experience - Gender', values='count')
df_charac = df_charac.fillna(0)
df_charac['sum'] = df_charac.sum(axis=1)
df_charac['Director Experience - Gender - Share of Males'] = df_charac['M'] / df_charac['sum']
df_charac['Director Experience - Gender - Share of Females'] = df_charac['F'] / df_charac['sum']
df_charac = df_charac.drop(columns=['F', 'M', 'sum'])
df_charac = df_charac.reset_index()
df_charac_gender = df_charac.copy()
df_charac_gender

Director Experience - Gender,CompanyID*,Year,Director Experience - Gender - Share of Males,Director Experience - Gender - Share of Females
0,270810,2002,0.933333,0.066667
1,270810,2003,0.875000,0.125000
2,270810,2004,0.875000,0.125000
3,270810,2005,0.812500,0.187500
4,270810,2006,0.800000,0.200000
...,...,...,...,...
44527,306519112781,2018,0.777778,0.222222
44528,306519112781,2019,0.777778,0.222222
44529,306519112781,2020,0.800000,0.200000
44530,307364612844,2019,0.875000,0.125000


In [19]:
# Get counts by nationality
df_charac = pd.DataFrame(df.groupby(['CompanyID*', 'Year', 'Director Experience - Nationality Mix'])['Director Experience - Nationality Mix'].count())
df_charac.columns = ['count']
df_charac = df_charac.reset_index()
df_charac = df_charac.pivot(index=['CompanyID*', 'Year'], columns='Director Experience - Nationality Mix', values='count')
df_charac = df_charac.fillna(0)

# Get most common nationalities, but return NaN if there is a tie
most_common_nationalities_var_list = ['Director Experience - Nationality Mix - Most Common Nationality']
# , 'Director Experience - Nationality Mix - 2nd Most Common Nationality', 'Director Experience - Nationality Mix - 3rd Most Common Nationality']
df_charac[most_common_nationalities_var_list] = df_charac.apply(lambda row: pd.Series(row.sort_values(ascending=False).index[:1]), axis=1)
# df_charac[most_common_nationalities_var_list] = df_charac.apply(lambda row: pd.Series(row.sort_values(ascending=False).index[:3]), axis=1)

# Get shares by nationality
df_charac['sum'] = df_charac.drop(columns=most_common_nationalities_var_list).sum(axis=1)
for charac in df_charac.columns.drop(['sum'] + most_common_nationalities_var_list):
    df_charac[f'Director Experience - Nationality Mix - Share of {charac}'] = df_charac[charac] / df_charac['sum']
    df_charac = df_charac.drop(columns=[charac])
df_charac = df_charac.drop(columns='sum')

df_charac = df_charac.reset_index()
df_charac_nationality = df_charac.copy()
df_charac_nationality

Director Experience - Nationality Mix,CompanyID*,Year,Director Experience - Nationality Mix - Most Common Nationality,Director Experience - Nationality Mix - Share of Algerian,Director Experience - Nationality Mix - Share of American,Director Experience - Nationality Mix - Share of Angolan,Director Experience - Nationality Mix - Share of Argentine,Director Experience - Nationality Mix - Share of Armenian,Director Experience - Nationality Mix - Share of Australian,Director Experience - Nationality Mix - Share of Austrian,Director Experience - Nationality Mix - Share of Bahraini,Director Experience - Nationality Mix - Share of Barbadian,Director Experience - Nationality Mix - Share of Belarusian,Director Experience - Nationality Mix - Share of Belgian,Director Experience - Nationality Mix - Share of Bermudian,Director Experience - Nationality Mix - Share of Bolivian,Director Experience - Nationality Mix - Share of Bosnian,Director Experience - Nationality Mix - Share of Brazilian,Director Experience - Nationality Mix - Share of British,Director Experience - Nationality Mix - Share of Bulgarian,Director Experience - Nationality Mix - Share of Burkinabe,Director Experience - Nationality Mix - Share of Burmese,Director Experience - Nationality Mix - Share of Canadian,Director Experience - Nationality Mix - Share of Caymanian,Director Experience - Nationality Mix - Share of Chilean,...,Director Experience - Nationality Mix - Share of Puerto Rican,Director Experience - Nationality Mix - Share of Qatari,Director Experience - Nationality Mix - Share of Romanian,Director Experience - Nationality Mix - Share of Russian,Director Experience - Nationality Mix - Share of Salvadoran,Director Experience - Nationality Mix - Share of Saudi,Director Experience - Nationality Mix - Share of Singaporean,Director Experience - Nationality Mix - Share of Slovak,Director Experience - Nationality Mix - Share of Slovene,Director Experience - Nationality Mix - Share of South African,Director Experience - Nationality Mix - Share of South Korean,Director Experience - Nationality Mix - Share of Spanish,Director Experience - Nationality Mix - Share of Sri Lankan,Director Experience - Nationality Mix - Share of Swedish,Director Experience - Nationality Mix - Share of Swiss,Director Experience - Nationality Mix - Share of Syrian,Director Experience - Nationality Mix - Share of Tanzanian,Director Experience - Nationality Mix - Share of Thai,Director Experience - Nationality Mix - Share of Turkish,Director Experience - Nationality Mix - Share of Ugandan,Director Experience - Nationality Mix - Share of Ukrainian,Director Experience - Nationality Mix - Share of Uruguayan,Director Experience - Nationality Mix - Share of Venezuelan,Director Experience - Nationality Mix - Share of Zambian,Director Experience - Nationality Mix - Share of Zimbabwean
0,270810,2002,American,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,270810,2003,American,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,270810,2004,American,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,270810,2005,American,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,270810,2006,American,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

# Get Company Details (bankers)

In [20]:
df_companydetails = pd.read_csv(r"Company Details.csv", low_memory=False)
df_companydetails

,CIK Code,HOAddress2,HOAddress1,Company Name,HOAddress4,Auditors,Latest AR,HO TelNumber,Bankers,Index,Ticker,Market Cap,ISIN,CompanyID*,HO FaxNumber,Sector,HOAddress5,HOCountryName,HO URL,HOAddress3,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name
0,NaN,Saint Helier,12 Castle Street,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Mar 2020,+44 (0)1534 711444,Citibank Europe PLC,FTSE 250,3IN,3040,JE00BF5FX167,9811997164,+44 (0)1534 609333,Investment Companies,JE2 3RT,Jersey,http://www.3i-infrastructure.com/,Channel Islands,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
1,NaN,Aarhus C,Christian Filtenborgs Plads 1,A/S SCHOUW & CO,Midtjylland,Ernst & Young Godkendt Revisionspartnerselskab...,Dec 2020,+45 86 11 22 22,NaN,NaN,SCHO,2321,DK0010253921,3552310896,+45 86 11 33 22,Diversified Industrials,DK-8000,Denmark,https://www.schouw.dk/en/home/,NaN,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
2,NaN,NaN,Via Lamarmora 230,A2A SPA,Brescia,EY SpA (Formerly known as Reconta Ernst & Youn...,Dec 2019,+39 030 35531,NaN,NaN,A2A,4883,IT0001233417,24982801,+39 030 355 3204,Electricity,25124,Italy,https://www.a2a.eu/en,NaN,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
3,NaN,Wiesbaden,Paulinenstrasse 15,AAREAL BANK AG,Hessen,KPMG AG Wirtschaftsprüfungsgesellschaft (Forme...,Dec 2019,+49 611 348 0,NaN,NaN,ARL,1476,DE0005408116,947432828,+49 611 348 25 49,Banks,D-65189,Germany,https://www.aareal-bank.com/en/,NaN,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
4,1091587.0,NaN,Affolternstrasse 44,ABB LTD,Zurich,KPMG AG,Dec 2020,+41 (0)43 317 7111,Citibank NA (Formerly known as First National ...,"OMX Stockholm 30, SMI , Stoxx Europe 50","ABB, ABBN",57423,CH0012221716,14501792,+41 (0)43 317 4420,Engineering & Machinery,CH-8050,Switzerland,https://global.abb/group/en,NaN,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310,1451554.0,NaN,1 Bedford Avenue,WILLIAM HILL PLC,London,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Dec 2020,+44 (0) 20 8918 3600,NaN,NaN,WMH,2892,GB0031698896,837912719,+44 (0) 20 8918 3775,Leisure & Hotels,WC1B 3AU,United Kingdom - England,https://www.williamhillplc.com/,NaN,UK - Company Details,UK,Director,Company Details,NaN,NaN
3311,NaN,Kennington Park Business Centre,Canterbury Court,WORKSPACE GROUP PLC (London Industrial PLC pri...,London,KPMG LLP,Mar 2020,+44 (0) 20 7138 3300,NatWest Markets PLC (Royal Bank of Scotland PL...,FTSE 250,WKP,1472,GB00B67G5X01,428697608,+44 (0) 20 7247 0157,Real Estate,SW9 6DE,United Kingdom - England,https://www.workspace.co.uk/,1-3 Brixton Road,UK - Company Details,UK,Director,Company Details,NaN,NaN
3312,NaN,25 Walbrook,The Walbrook Building,WORLDPAY GROUP PLC (De-listed 01/2018),London,NaN,Jan 2018,NaN,NaN,NaN,WPG,0,GB00BYYK2V80,24259147400,NaN,Business Services,EC4N 8AF,United Kingdom - England,http://www.worldpay.com/global,NaN,UK - Company Details,UK,Director,Company Details,NaN,NaN
3313,806968.0,18 Upper Ground,Sea Containers,WPP PLC (WPP Group prior to 11/2008),London,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Dec 2019,+44 (0) 20 7408 2204,Citibank NA (Formerly known as First National ...,FTSE 100,WPP,11568,JE00B8KF9B49,429577695,+44 (0) 20 7493 6819,Media & Entertainment,SE1 9GL,United Kingdom - England,https://www.wpp.com/,NaN,UK - Company Details,UK,Director,Company Details,NaN,NaN


In [24]:
df_companydetails.columns

Index(['CIK Code', 'HOAddress2', 'HOAddress1', 'Company Name', 'HOAddress4',
       'Auditors', 'Latest AR', 'HO TelNumber', 'Bankers', 'Index', 'Ticker',
       'Market Cap', 'ISIN', 'CompanyID*', 'HO FaxNumber', 'Sector',
       'HOAddress5', 'HOCountryName', 'HO URL', 'HOAddress3'],
      dtype='object')

In [21]:
df_companydetails = df_companydetails.drop_duplicates(['CompanyID*']).drop(columns=['inputfilestem', 'region', 'mgmt_type', 'workbook_name', 'fragment_number', 'sheet_name'])
df_companydetails.to_csv(r'C:\Users\jasonjia\Dropbox\projects\manager_and_board_characteristics\Board Charac_Jason\output\03_get_firm_and_manager_level_datasets\Company Details.csv', index=False)
df_companydetails

,CIK Code,HOAddress2,HOAddress1,Company Name,HOAddress4,Auditors,Latest AR,HO TelNumber,Bankers,Index,Ticker,Market Cap,ISIN,CompanyID*,HO FaxNumber,Sector,HOAddress5,HOCountryName,HO URL,HOAddress3
0,NaN,Saint Helier,12 Castle Street,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Mar 2020,+44 (0)1534 711444,Citibank Europe PLC,FTSE 250,3IN,3040,JE00BF5FX167,9811997164,+44 (0)1534 609333,Investment Companies,JE2 3RT,Jersey,http://www.3i-infrastructure.com/,Channel Islands
1,NaN,Aarhus C,Christian Filtenborgs Plads 1,A/S SCHOUW & CO,Midtjylland,Ernst & Young Godkendt Revisionspartnerselskab...,Dec 2020,+45 86 11 22 22,NaN,NaN,SCHO,2321,DK0010253921,3552310896,+45 86 11 33 22,Diversified Industrials,DK-8000,Denmark,https://www.schouw.dk/en/home/,NaN
2,NaN,NaN,Via Lamarmora 230,A2A SPA,Brescia,EY SpA (Formerly known as Reconta Ernst & Youn...,Dec 2019,+39 030 35531,NaN,NaN,A2A,4883,IT0001233417,24982801,+39 030 355 3204,Electricity,25124,Italy,https://www.a2a.eu/en,NaN
3,NaN,Wiesbaden,Paulinenstrasse 15,AAREAL BANK AG,Hessen,KPMG AG Wirtschaftsprüfungsgesellschaft (Forme...,Dec 2019,+49 611 348 0,NaN,NaN,ARL,1476,DE0005408116,947432828,+49 611 348 25 49,Banks,D-65189,Germany,https://www.aareal-bank.com/en/,NaN
4,1091587.0,NaN,Affolternstrasse 44,ABB LTD,Zurich,KPMG AG,Dec 2020,+41 (0)43 317 7111,Citibank NA (Formerly known as First National ...,"OMX Stockholm 30, SMI , Stoxx Europe 50","ABB, ABBN",57423,CH0012221716,14501792,+41 (0)43 317 4420,Engineering & Machinery,CH-8050,Switzerland,https://global.abb/group/en,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310,1451554.0,NaN,1 Bedford Avenue,WILLIAM HILL PLC,London,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Dec 2020,+44 (0) 20 8918 3600,NaN,NaN,WMH,2892,GB0031698896,837912719,+44 (0) 20 8918 3775,Leisure & Hotels,WC1B 3AU,United Kingdom - England,https://www.williamhillplc.com/,NaN
3311,NaN,Kennington Park Business Centre,Canterbury Court,WORKSPACE GROUP PLC (London Industrial PLC pri...,London,KPMG LLP,Mar 2020,+44 (0) 20 7138 3300,NatWest Markets PLC (Royal Bank of Scotland PL...,FTSE 250,WKP,1472,GB00B67G5X01,428697608,+44 (0) 20 7247 0157,Real Estate,SW9 6DE,United Kingdom - England,https://www.workspace.co.uk/,1-3 Brixton Road
3312,NaN,25 Walbrook,The Walbrook Building,WORLDPAY GROUP PLC (De-listed 01/2018),London,NaN,Jan 2018,NaN,NaN,NaN,WPG,0,GB00BYYK2V80,24259147400,NaN,Business Services,EC4N 8AF,United Kingdom - England,http://www.worldpay.com/global,NaN
3313,806968.0,18 Upper Ground,Sea Containers,WPP PLC (WPP Group prior to 11/2008),London,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Dec 2019,+44 (0) 20 7408 2204,Citibank NA (Formerly known as First National ...,FTSE 100,WPP,11568,JE00B8KF9B49,429577695,+44 (0) 20 7493 6819,Media & Entertainment,SE1 9GL,United Kingdom - England,https://www.wpp.com/,NaN


# Merge df with the additional variables

In [22]:
df_merge = df_groupby.merge(df_charac_gender, how='left', on=['CompanyID*', 'Year'])
df_merge = df_merge.merge(df_charac_nationality, how='left', on=['CompanyID*', 'Year'])
df_merge = df_merge.merge(df_companydetails, how='inner', on=['CompanyID*', 'ISIN', 'Sector', 'Company Name'])
df_merge

,Year,CompanyID*,Company Name,Sector,Country,ISIN,region,Characteristics of Roles - Director Network Size,Characteristics of Roles - Time to Retirement,Characteristics of Roles - Time in Role,Characteristics of Roles - Time on Board,Characteristics of Roles - Time in Company,Director Experience - Total Number of Quoted Boards to Date,Director Experience - Total Number of Private Boards to Date,Director Experience - Total Number of Other Boards to Date,Director Experience - Total Number of Quoted Current Boards,Director Experience - Total Number of Private Current Boards,Director Experience - Total Number of Other Current Boards,Director Experience - Avg. Yrs on Other Quoted Boards,Director Experience - Age (Yrs),Director Experience - Number of Qualifications,Ratios - Bonus/ (Bonus&Salary),Ratios - Equity Linked/ Total,Ratios - Performance/ Total,Ratios - %Change from Last Period,...,Director Experience - Nationality Mix - Share of Tanzanian,Director Experience - Nationality Mix - Share of Thai,Director Experience - Nationality Mix - Share of Turkish,Director Experience - Nationality Mix - Share of Ugandan,Director Experience - Nationality Mix - Share of Ukrainian,Director Experience - Nationality Mix - Share of Uruguayan,Director Experience - Nationality Mix - Share of Venezuelan,Director Experience - Nationality Mix - Share of Zambian,Director Experience - Nationality Mix - Share of Zimbabwean,CIK Code,HOAddress2,HOAddress1,HOAddress4,Auditors,Latest AR,HO TelNumber,Bankers,Index,Ticker,Market Cap,HO FaxNumber,HOAddress5,HOCountryName,HO URL,HOAddress3
0,2002,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,570.578947,7.857143,7.466667,9.972727,12.200000,1.545455,4.666667,1.000000,1.363636,1.888889,1.0,2.575000,59.428571,2.133333,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67716.0,Post Office Box 5650,1200 West Century Avenue,"North Dakota, ND",Deloitte & Touche LLP,Dec 2020,+1 701 530 1000,NaN,S&P MID CAP 400,MDU,6347,+1 866 760 4852,58506-5650,United States,https://www.mdu.com/,Bismarck
1,2003,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,596.809524,8.926667,6.693750,8.441667,11.162500,1.666667,4.615385,1.000000,1.333333,2.000000,1.0,3.415385,58.533333,2.062500,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67716.0,Post Office Box 5650,1200 West Century Avenue,"North Dakota, ND",Deloitte & Touche LLP,Dec 2020,+1 701 530 1000,NaN,S&P MID CAP 400,MDU,6347,+1 866 760 4852,58506-5650,United States,https://www.mdu.com/,Bismarck
2,2004,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,631.240000,9.420000,7.012500,8.715385,9.550000,1.666667,4.133333,1.000000,1.250000,1.750000,1.0,3.553846,58.066667,2.000000,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67716.0,Post Office Box 5650,1200 West Century Avenue,"North Dakota, ND",Deloitte & Touche LLP,Dec 2020,+1 701 530 1000,NaN,S&P MID CAP 400,MDU,6347,+1 866 760 4852,58506-5650,United States,https://www.mdu.com/,Bismarck
3,2005,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,589.965517,9.820000,5.543750,7.407692,8.525000,1.666667,4.266667,1.000000,1.333333,2.083333,1.0,4.323077,57.666667,1.875000,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67716.0,Post Office Box 5650,1200 West Century Avenue,"North Dakota, ND",Deloitte & Touche LLP,Dec 2020,+1 701 530 1000,NaN,S&P MID CAP 400,MDU,6347,+1 866 760 4852,58506-5650,United States,https://www.mdu.com/,Bismarck
4,2006,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,609.444444,10.485714,4.900000,7.054545,8.106667,1.636364,3.692308,1.000000,1.363636,2.333333,1.0,4.990909,56.857143,1.866667,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67716.0,Post Office Box 5650,1200 West Century Avenue,"North Dakota, ND",Deloitte & Touche LLP,Dec 2020,+1 701 530 1000,NaN,S&P MID CAP 400,MDU,6347,+1 866 760 4852,58506-5650,United States,https:/

# Cleanup df and save final df into csv

In [23]:
outputfilepath = Path(r"C:\Users\jasonjia\Dropbox\projects\manager_and_board_characteristics\Board Charac_Jason\output\03_get_firm_and_manager_level_datasets\boardex_firm_level_dataset.csv")
df_merge.to_csv(outputfilepath, index=False)